# Read the Source Tables


In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 2;


In [0]:
%sql
SELECT * FROM workspace.silver.erp_cust_az12 LIMIT 2;

In [0]:
%sql
SELECT * FROM workspace.silver.erp_loc_a101 LIMIT 2;

# Drop the table if already exists

In [0]:
%sql
DROP TABLE IF EXISTS workspace.gold.dim_customers

# Creat the Target Table

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold.dim_customers (customer_key INT, customer_id INT, customer_number STRING, customer_name STRING,  marital_status STRING, gender STRING, birthdate DATE, country STRING) USING DELTA


In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers

# Do the Transformations in the original table & insert to the target table

### Add Primary Key to the customers table , Join the erp_cust_az12 & erp_loc_a101 table

In [0]:
%sql
INSERT OVERWRITE workspace.gold.dim_customers
SELECT 
  ROW_NUMBER()OVER(ORDER BY ci.customer_id) AS customer_key,
  ci.customer_id,
  ci.customer_number,
  ci.customer_name,
  ci.marital_status,
  CASE 
    WHEN ci.gender !='n/a' THEN ci.gender
    ELSE COALESCE(ca.gender, 'n/a')
  END AS gender,
  ca.birthdate,
  cl.country 
FROM workspace.silver.crm_customers ci
LEFT JOIN workspace.silver.erp_cust_az12 ca
ON ci.customer_number = ca.customer_number
LEFT JOIN workspace.silver.erp_loc_a101 cl
ON ci.customer_number =  cl.customer_id

# Sanity Checks of Target Table

In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers